# M2177.003100 Deep Learning <br> Assignment #3 Part 3: Transformer

Copyright (C) Data Science Laboratory, Seoul National University. This material is for educational uses only. Some contents are based on the material provided by other paper/book authors and may be copyrighted by them. Written by Jeonghee Jo, October 2019, and modified by Jungbeom Lee, October 2020.

This is about Transformer (Vaswani et al., 2017).
https://papers.nips.cc/paper/7181-attention-is-all-you-need.pdf

Original blog post & code:
https://github.com/Kyubyong/transformer (Tensorflow)
https://pytorch.org/tutorials/beginner/transformer_tutorial.html (PyTorch)


That said, you are allowed to copy paste the codes from the original repo.
HOWEVER, <font color=red> try to implement the model yourself first </font>, and consider the original source code as a last resort.
You will learn a lot while wrapping around your head during the implementation. And you will understand nuts and bolts of Transformers more clearly in a code level.

### Submitting your work:
<font color=red>**DO NOT clear the final outputs**</font> so that TAs can grade both your code and results.  
Once you have done **all Assignment Part 1-3**, run the *CollectSubmission.sh* script with your **Student number** as input argument. <br>
This will produce a zipped file called *[Your student number].zip*. Please submit this file on ETL. &nbsp;&nbsp; (Usage: ./*CollectSubmission.sh* 20xx-xxxxx)


### Transformer (20 points)

This assignment is an on/off one: just make this notebook **"work"** without problem by: 

1. Explore various hyperparameters and pick the best set. You can change hyper-paraterers in the first code block. (10 points)
2. Please provide the analysis of changed hyper-parameters. (10 points)


### The Grading is as follows:

This assignment is an on/off one: just make this notebook **"work"** without problem by: 

1. Train your model using at least <font color=red> 12 different hyperparameter set </font>. Report performance results computed in the last code block <font color=red> for corresponding each hyperparameter set </font>. Plus, <font color=red> submit the one checkpoint file </font> of your best model. 

2. Please provide the analysis of changed hyper-parameters. 

The details are described in <font color=red>**transformer_modules.py**</font>. (There is nothing to implement in this notebook.)


Now proceed to the code.


In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
%cd drive/MyDrive/fastMRI_ming/Assignment3

/content/drive/MyDrive/fastMRI_ming/Assignment3


## Install libraries

In [3]:
import os
assert os.environ["CONDA_DEFAULT_ENV"] == "deep-learning-21", "current environment is not deep-learning-21"

KeyError: ignored

In [4]:
!python3 -m pip install torchtext==0.7

     |████████████████████████████████| 4.5 MB 13.6 MB/s 
     |████████████████████████████████| 1.2 MB 46.6 MB/s 
  Attempting uninstall: torchtext
    Found existing installation: torchtext 0.11.0
    Uninstalling torchtext-0.11.0:
      Successfully uninstalled torchtext-0.11.0


### Hyper-parameters

In [10]:
# change hyper-parameters in this code block!


emsize = 200 # embedding dimension
nhid = 100 # the dimension of the feedforward network model in nn.TransformerEncoder
nlayers = 2 # the number of nn.TransformerEncoderLayer in nn.TransformerEncoder
nhead = 2 # the number of heads in the multiheadattention models
dropout = 0.3 # the dropout value
batch_size = 32
eval_batch_size = 32
epochs = 20 # The number of epochs

### Load and Batch Data

In [11]:
%pip install torchtext==0.6.0
import torch
import torchtext
from torchtext.data.utils import get_tokenizer
TEXT = torchtext.data.Field(tokenize=get_tokenizer("basic_english"),
                            init_token='<sos>',
                            eos_token='<eos>',
                            lower=True)
train_txt, val_txt, test_txt = torchtext.datasets.WikiText2.splits(TEXT)
TEXT.build_vocab(train_txt)
%env CUDA_VISIBLE_DEVICES = 0
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
ntokens = len(TEXT.vocab.stoi) # the size of vocabulary
def batchify(data, bsz):
    data = TEXT.numericalize([data.examples[0].text])
    # Divide the dataset into bsz parts.
    nbatch = data.size(0) // bsz
    # Trim off any extra elements that wouldn't cleanly fit (remainders).
    data = data.narrow(0, 0, nbatch * bsz)
    # Evenly divide the data across the bsz batches.
    data = data.view(bsz, -1).t().contiguous()
    return data.to(device)



env: CUDA_VISIBLE_DEVICES=0


In [7]:
from transformer_modules import *
train_data = batchify(train_txt, batch_size)
val_data = batchify(val_txt, eval_batch_size)
test_data = batchify(test_txt, eval_batch_size)

bptt = 35
def get_batch(source, i):
    seq_len = min(bptt, len(source) - 1 - i)
    data = source[i:i+seq_len]
    target = source[i+1:i+1+seq_len].view(-1)
    return data, target
model = TransformerModel(ntokens, emsize, nhead, nhid, nlayers, dropout).to(device)

In [8]:
criterion = nn.CrossEntropyLoss()
lr = 5.0 # learning rate
optimizer = torch.optim.SGD(model.parameters(), lr=lr)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, 1.0, gamma=0.95)

import time
def train():
    model.train() # Turn on the train mode
    total_loss = 0.
    start_time = time.time()
    ntokens = len(TEXT.vocab.stoi)
    for batch, i in enumerate(range(0, train_data.size(0) - 1, bptt)):
        data, targets = get_batch(train_data, i)
        optimizer.zero_grad()
        output = model(data)
        loss = criterion(output.view(-1, ntokens), targets)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 0.5)
        optimizer.step()

        total_loss += loss.item()
        log_interval = 200
        if batch % log_interval == 0 and batch > 0:
            cur_loss = total_loss / log_interval
            elapsed = time.time() - start_time
            print('| epoch {:3d} | {:5d}/{:5d} batches | '
                  'lr {:02.2f} | ms/batch {:5.2f} | '
                  'loss {:5.2f} | ppl {:8.2f}'.format(
                    epoch, batch, len(train_data) // bptt, scheduler.get_lr()[0],
                    elapsed * 1000 / log_interval,
                    cur_loss, math.exp(cur_loss)))
            total_loss = 0
            start_time = time.time()

def evaluate(eval_model, data_source):
    eval_model.eval() # Turn on the evaluation mode
    total_loss = 0.
    ntokens = len(TEXT.vocab.stoi)
    with torch.no_grad():
        for i in range(0, data_source.size(0) - 1, bptt):
            data, targets = get_batch(data_source, i)
            output = eval_model(data)
            output_flat = output.view(-1, ntokens)
            total_loss += len(data) * criterion(output_flat, targets).item()
    return total_loss / (len(data_source) - 1)

In [12]:
best_val_loss = float("inf")

best_model = None

for epoch in range(1, epochs + 1):
    epoch_start_time = time.time()
    train()
    val_loss = evaluate(model, val_data)
    print('-' * 89)
    print('| end of epoch {:3d} | time: {:5.2f}s | valid loss {:5.2f} | '
          'valid ppl {:8.2f}'.format(epoch, (time.time() - epoch_start_time),
                                     val_loss, math.exp(val_loss)))
    print('-' * 89)

    if val_loss < best_val_loss:
        best_val_loss = val_loss
        best_model = model

    scheduler.step()
torch.save(best_model.state_dict(), 'models/Transformer.pth')

/usr/local/lib/python3.7/dist-packages/torch/optim/lr_scheduler.py:370: UserWarning: To get the last learning rate computed by the scheduler, please use `get_last_lr()`.
  "please use `get_last_lr()`.", UserWarning)


| epoch   1 |   200/ 7452 batches | lr 4.51 | ms/batch 22.22 | loss  5.86 | ppl   351.02
| epoch   1 |   400/ 7452 batches | lr 4.51 | ms/batch 21.59 | loss  5.82 | ppl   335.50
| epoch   1 |   600/ 7452 batches | lr 4.51 | ms/batch 21.59 | loss  5.73 | ppl   309.49
| epoch   1 |   800/ 7452 batches | lr 4.51 | ms/batch 21.62 | loss  5.70 | ppl   299.63
| epoch   1 |  1000/ 7452 batches | lr 4.51 | ms/batch 21.58 | loss  5.69 | ppl   295.72
| epoch   1 |  1200/ 7452 batches | lr 4.51 | ms/batch 21.55 | loss  5.82 | ppl   337.52
| epoch   1 |  1400/ 7452 batches | lr 4.51 | ms/batch 21.61 | loss  5.84 | ppl   343.25
| epoch   1 |  1600/ 7452 batches | lr 4.51 | ms/batch 21.57 | loss  5.72 | ppl   305.74
| epoch   1 |  1800/ 7452 batches | lr 4.51 | ms/batch 21.59 | loss  5.78 | ppl   324.82
| epoch   1 |  2000/ 7452 batches | lr 4.51 | ms/batch 21.61 | loss  5.73 | ppl   308.06
| epoch   1 |  2200/ 7452 batches | lr 4.51 | ms/batch 21.71 | loss  5.73 | ppl   307.38
| epoch   1 |  2400/ 

In [16]:
%ls


RNN.pth  Transformer.pth


In [ ]:
best_model_loaded = TransformerModel(ntokens, emsize, nhead, nhid, nlayers, dropout).to(device)
best_model_loaded.load_state_dict(torch.load('models/Transformer.pth'), strict=True)
test_loss = evaluate(best_model_loaded, test_data)
print('=' * 89)
print('| End of training | test loss {:5.2f} | test ppl {:8.2f}'.format(
    test_loss, math.exp(test_loss)))
print('=' * 89)

# Please provide your analysis on each hyper-parameter.
Epoch: 10 -> 20 ; Epoch를 증가시킬 때 loss가 더욱 감소되는 것을 알 수 있었습니다. gqu 성능으로 인하여 더 높은 epoch에 대해서 saturation되는 epoch의 값에 대해서는 구해볼 수 없었습니다. 

dropout: 0 -> 0.2 -> 0.3 ; 각 값에 대해서 진행해본 결과 0일 때 보다는 0.2에서 loss와 training time이 더 짧아지는 것을 알 수 있었습니다. 

n_layer: 1 -> 2 ; 1일 때보다 2에서 더 시간은 오래 걸렸지만 loss 값이 더 적게 나왔습니다. 

제가 구한 최적의 hyperparameter tuning 결과는 emsize = 200, nhid = 100, nlayers = 2, nhead = 2, dropout = 0.3, batch_size = 32, eval_batch_size = 32, epochs = 20 입니다. 